### Snippet example *was*

In [1]:
import numpy as np
import torch
import torch.nn as nn
import tensorflow as tf

In [2]:
def conv(attr_hs, attr_as, attr_vs, dim, feature_map_size=2, kernel_size=[2, 4], activation=tf.nn.tanh, layer_num=2):
    attr_as = tf.reshape(attr_as, [-1, 1, dim])
    attr_vs = tf.reshape(attr_vs, [-1, 1, dim])
    input_avs = tf.concat([attr_as, attr_vs], 1)
    input_shape = input_avs.shape.as_list()
    input_layer = tf.reshape(input_avs, [-1, input_shape[1], input_shape[2], 1])
    _conv = input_layer
    _conv = tf.layers.batch_normalization(_conv, 2)
    for i in range(layer_num):
        _conv = tf.layers.conv2d(inputs=_conv,
                                 filters=feature_map_size,
                                 kernel_size=kernel_size,
                                 strides=[1, 1],
                                 padding="same",
                                 activation=activation)
    _conv = tf.nn.l2_normalize(_conv, 2)
    _shape = _conv.shape.as_list()
    _flat = tf.reshape(_conv, [-1, _shape[1] * _shape[2] * _shape[3]])
    dense = tf.layers.dense(inputs=_flat, units=dim, activation=activation)
    dense = tf.nn.l2_normalize(dense)
    score = -tf.reduce_sum(tf.square(attr_hs - dense), 1)
    return score

### Snippet example *is*

In [6]:
class Conv(nn.Module):
	def __init__(self, sizes):
		super(Conv, self).__init__()
		self.norm_channels = sizes[3]
		self.in_channels = sizes[1]
		self.out_channels = sizes[2]
		self.con = nn.Conv2d(self.in_channels,
							 self.out_channels,
							 kernel_size=[2, 4])
		self.con_consequent = \
			nn.Conv2d(self.in_channels + 1,
					  self.out_channels,
					  kernel_size=[2, 4])
		self.nrm = nn.BatchNorm2d(self.norm_channels)
		self.activ_func_conv = activation()

	def forward(self, input, conv_times, pad=(2, 1, 1, 0)):

		sizes = list(input.shape)

		input = torch.reshape(input, [-1, sizes[3], sizes[2], 1])
		res = self.nrm(input)
		res = torch.reshape(res, [-1, 1, sizes[2], sizes[3]])

		padded = nn.functional.pad(res, pad)
		res = self.con(padded)
		res = self.activ_func_conv(res)

		for i in range(conv_times-1):  
			padded = nn.functional.pad(res, pad)
			res = self.con_consequent(padded)
			res = self.activ_func_conv(res)

		return res  

In [14]:
a = torch.randn(3, 75)
b = torch.randn(3, 75)
dim = 3

a = torch.reshape(a, [-1, 1, dim]) # 
b = torch.reshape(a, [-1, 1, dim])

aslist = list(c.shape)
c = torch.reshape(c, [-1, 1, aslist[1], aslist[0]])
def conv(attr_hs, attr_as, attr_vs, dim, feature_map_size=2, kernel_size=[2, 4], activation=nn.Tanh, layer_num=2):
    # print("feature map size", feature_map_size)
    # print("kernel size", kernel_size)
    # print("layer_num", layer_num)

    class Conv(nn.Module):
        def __init__(self, sizes):
            super(Conv, self).__init__()
            self.norm_channels = sizes[3]

            self.in_channels = sizes[1]
            self.out_channels = sizes[2]

            self.con = nn.Conv2d(self.in_channels, self.out_channels, kernel_size=[2, 4])
            self.con_consequent = nn.Conv2d(self.in_channels + 1, self.out_channels, kernel_size=[2, 4])
            self.nrm = nn.BatchNorm2d(self.norm_channels)

            self.activ_func_conv = activation()

        def forward(self, input, conv_times, pad=(2, 1, 1, 0)):

            sizes = list(input.shape)

            input = torch.reshape(input, [-1, sizes[3], sizes[2], 1])
            res = self.nrm(input)
            res = torch.reshape(res, [-1, 1, sizes[2], sizes[3]])

            padded = nn.functional.pad(res, pad)
            res = self.con(padded)
            res = self.activ_func_conv(res)

            for i in range(conv_times-1):
                padded = nn.functional.pad(res, pad)
                res = self.con_consequent(padded)
                res = self.activ_func_conv(res)

            return res

    attr_as = torch.reshape(attr_as, [-1, 1, dim])
    attr_vs = torch.reshape(attr_vs, [-1, 1, dim])

    input_avs = torch.cat([attr_as, attr_vs], 1)
    input_shape = list(input_avs.shape)
    input_layer = torch.reshape(input_avs, [-1, 1, input_shape[1], input_shape[0]])

    c = Conv(list(input_layer.shape))
    _conv = c.forward(input_layer, layer_num)

    _conv = nn.functional.normalize(_conv, dim=1, p=2)
    _shape = list(_conv.shape)
    _flat = torch.reshape(_conv, [-1, _shape[3] * _shape[2] * _shape[1]])
    # print("_flat", _flat.shape)

    class FlatProcessing(nn.Module):
        def __init__(self, in_, out_, activation_):
            super(FlatProcessing, self).__init__()
            self.pipeline = nn.Sequential(
                nn.Linear(in_, out_),
                activation_()
            )

        def forward(self, arg):
            return self.pipeline(arg)
    denser = FlatProcessing(_flat.shape[1], dim, activation_=activation)
    dense = denser.forward(_flat)
    dense = nn.functional.normalize(dense, dim=1, p=2)
    # print("dense", dense.shape)
    score = -torch.sum(torch.square(attr_hs - dense), 1)
    return score
print(conv(a,b,a,3))
print(a,b)

tensor([[ -6.3395, -11.4238,  -0.1997],
        [ -1.0331,  -6.3067,  -4.3848],
        [ -3.3284,  -6.7832,  -5.6104],
        [ -4.2825,  -2.9918,  -3.0187],
        [ -0.7765,  -5.8579,  -1.3900],
        [ -0.1474,  -6.7637,  -0.0690],
        [ -1.5665,  -1.5476,  -3.9487],
        [-30.9494,  -1.4184,  -5.9856],
        [ -8.4707,  -3.4516,  -0.6950],
        [ -0.1591,  -8.4773,  -9.4943],
        [ -1.2619,  -8.8456,  -2.2075],
        [-12.0580,  -5.1323,  -0.1489],
        [ -0.7447,  -2.0286,  -0.1908],
        [ -1.4702,  -1.8231,  -1.9911],
        [ -1.2020,  -7.6068,  -1.4482],
        [ -0.2710,  -1.4370,  -0.3663],
        [ -0.6182,  -2.0518,  -0.1240],
        [ -2.0925,  -5.3308,  -1.2050],
        [ -0.2407,  -4.2471,  -4.6557],
        [ -1.5930,  -1.4326,  -1.0387],
        [-28.1611,  -1.6681,  -2.9349],
        [ -1.9154,  -1.5121,  -0.1991],
        [ -0.3629,  -1.4179,  -1.2125],
        [ -0.1644,  -6.0255, -25.7708],
        [ -7.0197,  -1.6933,  -2.5137],
